In [50]:
# Importing all required methods
%matplotlib inline

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import myutils

In [51]:
# Directory where mp3 are stored.
AUDIO_DIR = os.environ.get('AUDIO_DIR')
current_directory = os.getcwd()

# Print the current working directory
print("Current directory:", current_directory)

# Load metadata and features.
tracks = myutils.load('data/fma_metadata/tracks.csv')
genres = myutils.load('data/fma_metadata/genres.csv')
features = myutils.load('data/fma_metadata/features.csv')
echonest = myutils.load('data/fma_metadata/echonest.csv')

Current directory: c:\Users\Tobey\Desktop\csc413-final-project


In [85]:
small_tracks = tracks[tracks['set', 'subset'] <= 'small']
small_tracks.shape

small_features = features[tracks['set', 'subset'] <= 'small']
small_features.shape

hottest_genres = small_tracks["track"]["genre_top"]

hottest_genres = hottest_genres.to_frame()

# for index, value in hottest_genres.iteritems():
#     print(f'Index: {index}, Value: {value}')

for column in hottest_genres.select_dtypes(include="category").columns:
    # print(f"Categories in {column}: {hottest_genres[column].cat.categories}")
    hottest_genres[column] = hottest_genres[column].cat.codes
    
print(hottest_genres[:3])

# hottest_genres = hottest_genres.dropna()

# print(len(hottest_genres))

# ipd.display(hottest_genres)

          genre_top
track_id           
2                 7
5                 7
10               12


In [ ]:
print('{} tracks, {} artists, {} albums, {} genres'.format(
    len(tracks), len(tracks['artist', 'id'].unique()),
    len(tracks['album', 'id'].unique()),
    sum(genres['#tracks'] > 0)))
mean_duration = tracks['track', 'duration'].mean()
print('track duration: {:.0f} days total, {:.0f} seconds average'.format(
    sum(tracks['track', 'duration']) / 3600 / 24,
    mean_duration))